In [4]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [5]:
from sklearn.pipeline import make_pipeline

In [6]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='/home/lfrolov/mlops-zoomcamp/artifacts/3', creation_time=1755101989007, experiment_id='3', last_update_time=1755101989007, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [7]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [8]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [9]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

/home/lfrolov/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2025/08/13 19:27:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2025/08/13 19:27:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run puzzled-squirrel-734 at: http://127.0.0.1:5000/#/experiments/3/runs/a8738c7e1ca64926bb35051d1c262e00
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


In [10]:
from mlflow.tracking import MlflowClient


In [11]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = 'b4d3bca8aa8e46a6b8257fe4541b1136'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [14]:
path = client.download_artifacts(run_id=RUN_ID, path='model')

In [23]:
with open(f'{path}MLmodel', 'rb') as f_out:
    dv = pickle.load(f_out)

UnpicklingError: unpickling stack underflow

In [22]:
dv

NameError: name 'dv' is not defined

In [20]:
!ls -la /tmp/tmp5nnwrlgp/model/

total 18684
drwxr-xr-x 2 lfrolov lfrolov     4096 Aug 13 19:30 .
drwx------ 3 lfrolov lfrolov     4096 Aug 13 19:30 ..
-rw-r--r-- 1 lfrolov lfrolov      677 Aug 13 19:27 MLmodel
-rw-r--r-- 1 lfrolov lfrolov      223 Aug 13 19:27 conda.yaml
-rw-r--r-- 1 lfrolov lfrolov 19104434 Aug 13 19:27 model.pkl
-rw-r--r-- 1 lfrolov lfrolov      120 Aug 13 19:27 python_env.yaml
-rw-r--r-- 1 lfrolov lfrolov      104 Aug 13 19:27 requirements.txt
